Imports:

In [19]:
import os
import pandas as pd
import re
import numpy as np

Filenames and paths:

In [20]:
#path for input and output
inputpath = '/Users/pohno/Desktop/dad_data'
outputpath = '/Users/pohno/Desktop/dad_data'

#filepath and name for metabolite file
f_met = inputpath + '/' + 'sampleTransformationFile.txt'

#filepath and name for peak lists
f1name = inputpath +'/' + 'Time0.xlsx'
f2name = inputpath + '/' + 'Time5.xlsx'

Import metabolite list and calculate element columns to match format:

In [21]:
#import dataframe of metabolites
df_met = pd.read_csv(f_met,sep="\t")
df_met.rename(columns = {'no':'Formula'}, inplace = True) 

#add columns for each element
df_met['C'] = pd.Series([0] * len(df_met.index))
df_met['H'] = pd.Series([0] * len(df_met.index))
df_met['N'] = pd.Series([0] * len(df_met.index))
df_met['O'] = pd.Series([0] * len(df_met.index))
df_met['S'] = pd.Series([0] * len(df_met.index))
df_met['P'] = pd.Series([0] * len(df_met.index))

#iterate through dataframe (should vectorize for better performance) and populate element columns
#turn off warning
pd.options.mode.chained_assignment = None  # default='warn'
for row in range(len(df_met.index)):
    a = re.findall(r'([A-Z][a-z]*)(\d*)', df_met['Formula'][row])
    for b in a:
        if b[1] == '':
            df_met[b[0]][row] = 1
        else:
            df_met[b[0]][row] = int(b[1])
print('Metabolites:')
print(df_met)

Metabolites:
                  Isomeric    Formula           0   C   H  N   O  S  P
0                  glucose    C6H12O6  179.055565   6  12  0   6  0  0
1                  glucose    C6H12O6  162.052825   6  12  0   6  0  0
2                  pentose     C5H8O4  132.042253   5   8  0   4  0  0
3                  Alanine     C3H5NO   71.037114   3   5  1   1  0  0
4                 Arginine   C6H12N4O  156.101111   6  12  4   1  0  0
..                     ...        ...         ...  ..  .. ..  .. .. ..
83  Glucuronic Acid (-H2O)     C6H8O6  176.032088   6   8  0   6  0  0
84   monosaccharide (-H2O)    C6H10O5  162.052824   6  10  0   5  0  0
85    trisaccharide (-H2O)  C18H30O15  486.158471  18  30  0  15  0  0
86         erythose (-H2O)     C4H8O4  102.031695   4   8  0   4  0  0
87     transamination (-O)        NH4    2.039459   0   4  1   0  0  0

[88 rows x 9 columns]


Do subtraction and comparison:

In [22]:
#import two peak lists
df1 =  pd.read_excel(f1name)
df2 =  pd.read_excel(f2name)

#output file
f_out_name = outputpath + '/' + 'CountForEachMetabolite.csv'
f_out = open(f_out_name,'w')
f_out.write('Name,Count\n')

#slice dataframes into only columns to compare
df1comp = df1.loc[:, ['C', 'H', 'N','O','S','P']]
df2comp = df1.loc[:, ['C', 'H', 'N','O','S','P']]
df_metcomp = df_met.loc[:, ['C', 'H', 'N','O','S','P']]

for row in range(len(df_met.index)):
    met = df_metcomp.loc[row]
    df1sub = df1comp-met
    df = pd.merge(df1sub, df2comp, how='left', indicator='Exist')
    df['Exist'] = np.where(df.Exist == 'both', True, False)
    matches = df1[df.Exist==True]
    print(matches)
    name = outputpath + '/met' + str(row) + '_' + df_met['Isomeric'][row] + '.csv'
    matches.to_csv(name)
    print(df_met.loc[row][0],'\t',len(matches.index))
    f_out.write(str(df_met.loc[row][0])+','+str(len(matches.index))+'\n')
   
f_out.close()

       C   H  N   O  S  P     Intensity
1114  20  22  0  15  0  0  7.169798e+06
1138  20  24  0  14  0  0  8.440787e+06
1139  20  24  0  15  0  0  3.828454e+06
1158  20  26  0  13  0  0  7.545890e+06
1159  20  26  0  14  0  0  3.064757e+06
...   ..  .. ..  .. .. ..           ...
3864  20  30  0  17  0  0  2.526731e+06
3871  26  42  0  23  0  0  2.814148e+06
3872  28  46  0  24  0  0  3.465196e+06
3874  30  48  0  26  0  0  2.518430e+06
3882  23  36  0  17  0  0  2.411960e+06

[817 rows x 7 columns]
glucose 	 817
       C   H  N   O  S  P     Intensity
1114  20  22  0  15  0  0  7.169798e+06
1138  20  24  0  14  0  0  8.440787e+06
1139  20  24  0  15  0  0  3.828454e+06
1158  20  26  0  13  0  0  7.545890e+06
1159  20  26  0  14  0  0  3.064757e+06
...   ..  .. ..  .. .. ..           ...
3864  20  30  0  17  0  0  2.526731e+06
3871  26  42  0  23  0  0  2.814148e+06
3872  28  46  0  24  0  0  3.465196e+06
3874  30  48  0  26  0  0  2.518430e+06
3882  23  36  0  17  0  0  2.411960e+06

[